In [2]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from groq import Groq
import os

In [7]:
GROQ_API_KEY= os.getenv('GROQ_API_KEY')

In [8]:
os.getcwd()

'/Users/pranayyb/Desktop/dateher/DateHer'

In [224]:
loader = DirectoryLoader("/Users/pranayyb/Desktop/dateher/DateHer/", glob = "./*.txt", loader_cls= TextLoader)
document=loader.load()

In [226]:
document

[Document(metadata={'source': '/Users/pranayyb/Desktop/dateher/DateHer/requirements.txt'}, page_content='fastapi\nuvicorn\npydantic\ntensorflow\nflask\nflask-cors\nhttpx\npandas\nnltk\nscikit-learn\nrequests\ngensim\ntextblob\nspacy\nastrapy'),
 Document(metadata={'source': '/Users/pranayyb/Desktop/dateher/DateHer/Dating.txt'}, page_content='\ufeff"Is my girlfriend or boyfriend cheating on me?"\nThis is the absolute, number one relationship question of all time. If you go to Google and type in “Is my wife,” or “Is my husband,” Google will finish your sentence with “Cheating on me”. Sometimes, you don’t even get past the “my”!\nUnfortunately, science can’t provide a definitive answer to the question of cheating. It can, however, give us the tools to spot deception.\nDr. Paul Ekman, University of California Medical School emeritus psychology professor, has published something called the Facial Action Coding System—FACS—which successfully detects deception close to 90 percent of the time.

In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
text = text_splitter.split_documents(document)

In [18]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/var/folders/8s/589js3cj3k5gf6dtzp4t6v7r0000gn/T/ipykernel_7909/3409896792.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [19]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [20]:
from astrapy import DataAPIClient
from astrapy.constants import VectorMetric

In [21]:
client = DataAPIClient(os.getenv("ASTRA_DB_APPLICATION_TOKEN"))
database = client.get_database(os.getenv("ASTRA_DB_API_ENDPOINT"))

In [22]:
collection = database.create_collection(
    "dateher_chat",
    dimension=384,
    metric=VectorMetric.COSINE,  # Or just 'cosine'.
    check_exists=True, # Optional.
)

In [23]:
text[170]

Document(metadata={'source': '/Users/pranayyb/Desktop/dateher/DateHer/Dating.txt'}, page_content="3. Effective communication involves active listening and expressing your thoughts clearly. Use 'I' statements to articulate your feelings and needs. Focus on discussing the issue at hand rather than attacking your partner, which can help maintain a positive atmosphere.\n________________")

In [24]:
chunk_embeddings = [embeddings.embed_query(chunk.page_content) for chunk in text]

In [25]:
chunk_embeddings

[[-0.0868706926703453,
  -0.02745177410542965,
  -0.06899623572826385,
  0.06269524991512299,
  0.06564301252365112,
  -0.12504057586193085,
  -0.023989470675587654,
  -0.034133974462747574,
  -0.05529129132628441,
  -0.09531326591968536,
  0.068677619099617,
  -0.05971936881542206,
  -0.09026585519313812,
  -0.03544672951102257,
  0.018298976123332977,
  0.016388101503252983,
  -0.027040597051382065,
  0.01070016622543335,
  0.005995820742100477,
  -0.06795532256364822,
  -0.04135729372501373,
  0.07936359196901321,
  0.0825502797961235,
  0.036201998591423035,
  -0.004757851827889681,
  -0.07001818716526031,
  -0.059278205037117004,
  -0.041395243257284164,
  0.050726328045129776,
  -0.03707166016101837,
  0.021678293123841286,
  0.09075440466403961,
  -0.02120964229106903,
  0.06526842713356018,
  -0.024336257949471474,
  0.05931134149432182,
  -0.03839462623000145,
  -0.09380067139863968,
  0.05921680107712746,
  0.06688772141933441,
  0.0021282292436808348,
  0.00863746926188469,


In [28]:
documents=[]

In [31]:
for i, chunk in enumerate(chunk_embeddings):
    piece = {
        "text": text[i].page_content,
        "$vector": chunk
    }
    documents.append(piece)


In [32]:
documents[78]

{'text': "Conflict Resolution - Revised Patterns:\n                              1. I believe in discussing problems openly.\nThat’s a constructive approach! Open discussions can foster understanding. How do you initiate these conversations to ensure they’re productive?\n                              2. I tend to steer clear of conflicts whenever possible.\nInteresting! Avoiding conflicts can sometimes maintain peace. How do you manage situations where avoidance isn't possible?\n                              3. I confront issues head-on when they arise.\nAbsolutely! Addressing issues directly shows your commitment to clarity. What strategies do you use to ensure these conversations are effective?\n                              4. I appreciate constructive criticism during disagreements.\nWonderful! Being open to feedback can lead to growth. How do you approach receiving feedback when tensions are high?\n                              5. I try to navigate disagreements thoughtfully.",
 '

In [33]:
insertion_result = collection.insert_many(documents)

In [34]:
print(f"* Database: {database.info().name}\n")

* Database: hehe



In [35]:
print(f"* Collection: {collection.full_name}\n")

* Collection: default_keyspace.dateher_chat



In [190]:
from langchain_core.runnables import RunnableLambda

class GroqLLM(RunnableLambda):
    def __init__(self, api_key, model_name):
        self.client = Groq(api_key=api_key)
        self.model_name = model_name

    def invoke(self, *args, **kwargs):
        # Check if we have retrieved texts in args
        if args and isinstance(args[0], list) and all(isinstance(text, str) for text in args[0]):
            # Combine retrieved texts into a single string
            combined_texts = "\n".join(args[0])
        else:
            raise ValueError("Input texts are required and must be a list of strings.")

        stop = kwargs.get('stop', None)

        # Ensure combined_texts is a string
        if combined_texts is None or not isinstance(combined_texts, str):
            raise ValueError("Input texts are required and must be a string.")

        # Create the chat completion request
        chat_completion = self.client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a dating chatbot and you are chatting as a real person, suggesting and answering their queries as needed. If you have no answer, then simply generate one of your own, but be relevant to it. juset use plain text no bold italics etc, if no answer seems relevant just ask them to keep queries related to dating only...stop using of course"
                },
                {
                    "role": "user",
                    "content": combined_texts + "\n\nBased on the above information, answet the user query to help them"
                }
            ],
            model=self.model_name,
        )

        response = chat_completion.choices[0].message.content

        # Handle stop tokens if provided
        if stop and response:
            for token in stop:
                if token in response:
                    response = response.split(token)[0]

        return response


In [194]:
model_name="mixtral-8x7b-32768"
model = GroqLLM(api_key=GROQ_API_KEY, model_name=model_name)

In [198]:
def get_relevant_documents(query):
    query_vector = embeddings.embed_query(query)
    results = collection.find(
        sort={"$vector": query_vector},
        limit=30,
        include_similarity=True,
    )
    
    relevant_documents = [doc for doc in results if doc["$similarity"] >= 0.7]
    
    return relevant_documents

In [208]:
def gen(query):
    res=get_relevant_documents(query)
    docc=[]
    f=0
    for i in res:
        if(f>10):
            break
        docc.append(i['text'])
        f=f+1
    response = model.invoke(docc)
    return response

In [216]:
response=gen("How to start a conversation?")

In [217]:
print(response)

To start a conversation with a match, consider asking an open-ended question related to their interests or experiences. This can help invite discussion and create a connection. For example, if they mention liking a particular movie or book, you could ask what drew them to it and what they enjoyed most about it. Remember, personalization is key when initiating a conversation.

If you need more help or want specific examples tailored to your situation, feel free to ask!


In [218]:
gen("what is hyundai?")

"Of course! I'm here to help with your dating-related queries. What would you like to know or discuss about dating? If you have any questions or need advice, feel free to ask. If I don't have an appropriate answer, I'll let you know and ask you to keep the queries related to dating. Go ahead!"

In [279]:
from langchain.prompts import PromptTemplate

clt = Groq(
    api_key=GROQ_API_KEY,
)
prompt_template = PromptTemplate(
    input_variables=["text", "attribute_type"],
    template="Extract a list of {attribute_type} mentioned in the following text, ensuring only positive or neutral attributes are included (ignore dislikes or negative mentions): {text}. Provide only the words in a Python list, without any additional text.",
)

formatted_prompt = prompt_template.format(text=input_text, attribute_type=attribute_type)

response = clt.chat.completions.create(
    messages=[{"role": "user", "content": formatted_prompt}],  
    model="llama3-8b-8192",  
)

output = response.choices[0].message.content
try:
    # Remove any leading/trailing whitespace and additional text if needed
    cleaned_output = output.strip()
    
    # Use regex to match and extract the content within square brackets
    match = re.search(r'\[.*\]', cleaned_output)
    if match:
        # Convert the matched string to a list
        words_list = eval(match.group(0))  # Use eval to convert the string representation to a list
        print(words_list)  # Print the result list
    else:
        print("No valid list found in the output.")
except Exception as e:
    print(f"Error processing output: {e}")

['enjoy', 'love', 'excites']


In [271]:

# Safely convert the string representation of the list to a Python list
try:
    words_list = ast.literal_eval(output)
    print(words_list)
except Exception as e:
    print(f"Error processing output: {e}")

Error processing output: invalid syntax (<unknown>, line 1)


In [281]:
def char(input_text, attribute_type):
    formatted_prompt = prompt_template.format(
        text=input_text, attribute_type=attribute_type
    )
    response = clt.chat.completions.create(
        messages=[{"role": "user", "content": formatted_prompt}],
        model="llama3-8b-8192",
    )
    output = response.choices[0].message.content
    try:
        # Remove any leading/trailing whitespace and additional text if needed
        cleaned_output = output.strip()
        # Use regex to match and extract the content within square brackets
        match = re.search(r"\[.*\]", cleaned_output)
        if match:
            # Convert the matched string to a list
            words_list = eval(
                match.group(0)
            )  # Use eval to convert the string representation to a list
            print(words_list)  # Print the result list
        else:
            print("No valid list found in the output.")
    except Exception as e:
        print(f"Error processing output: {e}")

In [283]:
char("I enjoy painting and love spending time in nature. Traveling to new places excites me.","interests")

['painting', 'nature', 'traveling']


In [ ]:
input_text = "I enjoy painting and love spending time in nature. Traveling to new places excites me."
attribute_type = "traits"